In [1]:
import os
import cv2
import numpy as np

## 1. Parâmetros

In [2]:
# Caminho para o diretório de importação de imagens
import_dir = os.path.join( ".", "archive", "images", "images" )

# Caminho para o diretório de salvamento de imagens
export_dir = os.path.join( ".", "data" )

# Número de exemplos por classe de cada partição do banco de dados
num_of_test_images_per_class = 500
num_of_train_images_per_class = 1000
num_of_val_images_per_class = num_of_test_images_per_class

# Dicionário que associa números inteiros e classes
class_dict = { 0: "Cachorro", 1: "Gato" }

## 2. Seleção de Imagens de Teste e Validação

In [3]:
path_to_test_file = os.path.join( ".", "archive", "annotations", "annotations", "test.txt" )
assert os.path.exists(path_to_test_file), "O txt referente aos exemplos de teste não foi encontrado!"

# Lista os caminhos de imagens da classe "Gato"
test_cats = []

# Lista os caminhos de imagens da classe "Cachorro"
test_dogs = []

# Abre o arquivo de texto
with open(path_to_test_file) as f:
    
    # Lê as linhas do arquivo de texto
    lines = f.readlines()
    
    # Itera as linhas do arquivo
    for idx, line in enumerate(lines):
        
        # Seleciona tudo antes do primeiro espaço na linha lida e adiciona a extensão
        filename = line.split(" ")[0]+".jpg"
        
        # Combina o diretório de importação e o nome da imagem para produzir o caminho completo
        import_path = os.path.join( import_dir, filename )
        
        # Verifica se a primeira letra do arquivo é maiúscula ou não
        if filename[0].isupper():
            # Imagens com 1º letra maiúscula são imagens de gatos
            # Adiciona o caminho à lista de gatos
            test_cats.append( import_path )
        else:
            # Imagens com 1º letra minúscula são imagens de cachorros
            # Adiciona o caminho à lista de cachorros
            test_dogs.append( import_path )

# Embaralha os elementos das duas listas - Uma mesma semente é configurada para produzir sempre os mesmos conjuntos
np.random.seed(42)
np.random.shuffle(test_cats)
np.random.seed(42)
np.random.shuffle(test_dogs)

# Seleciona os primeiros elementos das listas após embaralhar os elementos para o conjunto de validação
# O número de elementos selecionados é dado por num_of_val_images_per_class
validation_cats = test_cats[:num_of_val_images_per_class]
validation_dogs = test_dogs[:num_of_val_images_per_class]

# Seleciona os primeiros elementos das listas após os elementos selecionados para o conjunto de validação
# O número de elementos selecionados é dado por num_of_test_images_per_class
test_cats = test_cats[num_of_val_images_per_class:(num_of_val_images_per_class + num_of_test_images_per_class)]
test_dogs = test_dogs[num_of_val_images_per_class:(num_of_val_images_per_class + num_of_test_images_per_class)]

# A soma de listas equivale à sua concatenação (isso junta as duas listas em uma só)
test_cats_and_dogs = test_cats + test_dogs
validation_cats_and_dogs = validation_cats + validation_dogs

# Printa o número de exemplos em cada conjunto
print("{} Imagens selecionadas para teste, {} cães e {} gatos.".format( len(test_cats_and_dogs),  
                                                                        len(test_dogs),  
                                                                        len(test_cats) ))

print("{} Imagens selecionadas para validação, {} cães e {} gatos.".format( len(validation_cats_and_dogs),  
                                                                            len(validation_dogs),  
                                                                            len(validation_cats) ))

1000 Imagens selecionadas para teste, 500 cães e 500 gatos.
1000 Imagens selecionadas para validação, 500 cães e 500 gatos.


## 3. Seleção de Imagens de Treino

In [4]:
path_to_train_file = os.path.join( ".", "archive", "annotations", "annotations", "trainval.txt" )
assert os.path.exists(path_to_train_file), "O txt referente aos exemplos de treino não foi encontrado!"

# Lista os caminhos de imagens da classe "Gato"
train_cats = []

# Lista os caminhos de imagens da classe "Cachorro"
train_dogs = []

# Abre o arquivo de texto
with open(path_to_train_file) as f:
    
    # Lê as linhas do arquivo de texto
    lines = f.readlines()
    
    # Itera as linhas do arquivo
    for idx, line in enumerate(lines):
        
        # Seleciona tudo antes do primeiro espaço na linha lida e adiciona a extensão
        filename = line.split(" ")[0]+".jpg"
        
        # Combina o diretório de importação e o nome da imagem para produzir o caminho completo
        import_path = os.path.join( import_dir, filename )
        
        # Verifica se a primeira letra do arquivo é maiúscula ou não
        if filename[0].isupper():
            # Imagens com 1º letra maiúscula são imagens de gatos
            # Adiciona o caminho à lista de gatos
            train_cats.append( import_path )
        else:
            # Imagens com 1º letra minúscula são imagens de cachorros
            # Adiciona o caminho à lista de cachorros
            train_dogs.append( import_path )

# Embaralha os elementos das duas listas - Uma mesma semente é configurada para produzir sempre os mesmos conjuntos
np.random.seed(42)
np.random.shuffle(train_cats)
np.random.seed(42)
np.random.shuffle(train_dogs)

# Seleciona os primeiros elementos das listas após embaralhar os elementos para o conjunto de treino
# O número de elementos selecionados é dado por num_of_train_images_per_class
train_cats = train_cats[:num_of_train_images_per_class]
train_dogs = train_dogs[:num_of_train_images_per_class]

# A soma de listas equivale à sua concatenação (isso junta as duas listas em uma só)
train_cats_and_dogs = train_cats + train_dogs

print("{} Imagens selecionadas para treino, {} cães e {} gatos.".format( len(train_cats_and_dogs),  
                                                                         len(train_dogs),  
                                                                         len(train_cats) ))

2000 Imagens selecionadas para treino, 1000 cães e 1000 gatos.


## 4. Salvamento das imagens

In [5]:
partition_list = ["treino", "validação", "teste"]
paths_per_partition_list = [train_cats_and_dogs, validation_cats_and_dogs, test_cats_and_dogs]

for partition, path_list in zip( partition_list, paths_per_partition_list ):
    print("\n{} partition: {} paths".format( partition, len(path_list) ))
    
    # Itera as linhas do arquivo
    for idx, path in enumerate(path_list):
        
        print("\tImagem {}/{}".format( str(idx).zfill(4), len(path_list) ), end = "\r")
        
        # Identifica o nome do arquivo a partir do caminho
        filename = os.path.basename( path )
        
        # Verifica se a primeira letra do arquivo é maiúscula ou não
        # Imagens com 1º letra maiúscula são imagens de gatos
        # Imagens com 1º letra minúscula são imagens de cachorros
        classe = 1 if filename[0].isupper() else 0
        
        # Produz o caminho de salvamento para as imagens a partir do diretório base e da classe
        # Imagens de gato serão salvos em "./data/<partition>/gato/"
        # Imagens de cachorros serão salvos em "./data/<partition>/cachorro/"
        export_dirname = os.path.join( export_dir, partition, class_dict[classe] )
        
        # Cria o diretório de salvamento caso ele ainda não exista
        if not os.path.exists( export_dirname ):
            os.makedirs( export_dirname )
        
        # Combina o diretório de salvamento e o nome da imagem para produzir o caminho completo
        export_path = os.path.join( export_dirname, filename )
        
        # Carrega a imagem
        src_img = cv2.imread( path )
        
        # Redimensiona a imagem para 150 x 150
        src_img = cv2.resize( src_img, (150, 150) )
        
        # Salva a imagem redimensionada
        cv2.imwrite(export_path, src_img)


treino partition: 2000 paths
	Imagem 1999/2000
validação partition: 1000 paths
	Imagem 0999/1000
teste partition: 1000 paths


In [7]:
partition = "validação"

# Itera as linhas do arquivo
for idx, path in enumerate(validation_cats_and_dogs):

    print("\tImagem {}/{}".format( str(idx).zfill(4), len(path_list) ), end = "\r")

    # Identifica o nome do arquivo a partir do caminho
    filename = os.path.basename( path )

    # Verifica se a primeira letra do arquivo é maiúscula ou não
    # Imagens com 1º letra maiúscula são imagens de gatos
    # Imagens com 1º letra minúscula são imagens de cachorros
    classe = 1 if filename[0].isupper() else 0

    # Produz o caminho de salvamento para as imagens a partir do diretório base e da classe
    # Imagens de gato serão salvos em "./data/<partition>/gato/"
    # Imagens de cachorros serão salvos em "./data/<partition>/cachorro/"
    export_dirname = os.path.join( export_dir, partition, class_dict[classe] )

    # Cria o diretório de salvamento caso ele ainda não exista
    if not os.path.exists( export_dirname ):
        os.makedirs( export_dirname )

    # Combina o diretório de salvamento e o nome da imagem para produzir o caminho completo
    export_path = os.path.join( export_dirname, filename )

    # Carrega a imagem
    src_img = cv2.imread( path )

    # Redimensiona a imagem para 150 x 150
    src_img = cv2.resize( src_img, (150, 150) )

    # Salva a imagem redimensionada
    cv2.imwrite(export_path, src_img)

In [8]:
export_path

'.\\data\\validação\\Cachorro\\pomeranian_66.jpg'